# VAE Training - Faces dataset

## imports

In [1]:
import tensorflow as tf

# from google.colab import drive
# drive.mount('/content/drive')

tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)

In [2]:
#  tf.test.gpu_device_name()

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
# !cat /proc/meminfo


In [5]:
# !cat /proc/cpuinfo


In [6]:
# memory footprint support libraries/code

# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

In [7]:
# cd '/content/drive/My Drive/Books&Code/GenerativeDeepLearning'


In [8]:
# !cp data/img_align_celeba.zip /content/.

In [9]:
# !unzip /content/img_align_celeba.zip

In [10]:
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [11]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'

RUN_FOLDER = 'run/{}/'.format(section)
# RUN_FOLDER = '/content/run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = './data/img_align_celeba/'
# DATA_FOLDER = '/content/img_align_celeba/'

## data

In [12]:
os.path.join(DATA_FOLDER, '*/*.jpg')

'./data/img_align_celeba/*/*.jpg'

In [13]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 256

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [14]:
NUM_IMAGES

202599

In [15]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 202599 images belonging to 1 classes.


## architecture

In [16]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [17]:
vae.encoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [18]:
vae.decoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0   

## training

In [19]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [20]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [21]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
792/791 [==============================] - 229s 289ms/step - loss: 351.7912 - vae_r_loss: 280.9302 - vae_kl_loss: 70.9312

Epoch 00001: saving model to run/vae/0001_faces/weights/weights-001-351.86.h5

Epoch 00001: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 2/200
792/791 [==============================] - 221s 279ms/step - loss: 246.4389 - vae_r_loss: 186.8766 - vae_kl_loss: 59.5641

Epoch 00002: saving model to run/vae/0001_faces/weights/weights-002-246.44.h5

Epoch 00002: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 3/200
792/791 [==============================] - 200s 252ms/step - loss: 234.1830 - vae_r_loss: 175.3955 - vae_kl_loss: 58.7856

Epoch 00003: saving model to run/vae/0001_faces/weights/weights-003-234.18.h5

Epoch 00003: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 4/200
792/791 [==============================] - 181s 229ms/step - loss: 228.8476 - vae_r_loss: 170.1213 - vae_kl_loss: 58.7318

Epoch 00004: saving mo